<a href="https://colab.research.google.com/github/tarupathak30/Deep-Learning-/blob/main/LSTM/next_word_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
mylesoneill_classic_literature_in_ascii_path = kagglehub.dataset_download('mylesoneill/classic-literature-in-ascii')
ronikdedhia_next_word_prediction_path = kagglehub.dataset_download('ronikdedhia/next-word-prediction')
ysthehurricane_next_word_prediction_bi_lstm_tutorial_easy_way_path = kagglehub.notebook_output_download('ysthehurricane/next-word-prediction-bi-lstm-tutorial-easy-way')

print('Data source import complete.')


In [ ]:
# file_path = '/kaggle/input/classic-literature-in-ascii/NONFICTION/NONFICTION/bacon-new-98.txt'

In [ ]:
# import os

# # Read the text
# with open(file_path, 'r', encoding='utf-8') as f:
#     text = f.read()


In [ ]:
import os

folder_path = '/kaggle/input/classic-literature-in-ascii/'

print("Exists:", os.path.exists(folder_path))
print("Contents:", os.listdir(folder_path))

Exists: True
Contents: ['NONFICTION', 'REFERENCE', 'FICTION', 'AUTHORS']


In [ ]:
folder_path = '/kaggle/input/classic-literature-in-ascii/NONFICTION/NONFICTION'
all_texts = []

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8', errors='ignore') as f:
            all_texts.append(f.read())

text = " ".join(all_texts)
print(text[:500])


                                     500 BC
                                BUDDHA, THE WORD
                              (The Eightfold Path)

                         THE FOUR NOBLE TRUTHS

  THUS has it been said by the Buddha, the Enlightened One: It is
through not understanding, not realizing four things, that I,
Disciples, as well as you, had to wander so long through this round of
rebirths. And what are these four things? They are the Noble Truth
of Suffering, the Noble Truth of the Orig


In [ ]:
corpus = text

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

2025-09-21 21:16:19.092692: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758489379.462022      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758489379.561664      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
corpus[8000:10000]

', or whether it be the external\nfluid element, they are both only the fluid element.\n  And one should understand, according to reality, and true wisdom:\n"This does not belong to me; this am I not; this is not my Ego."\n  3. What, now, is the Heating Element? The heating element may be one\nown, or it may be external. And what is one\'s own heating element? The\ndependent properties, which on one\'s own person and body are heating\nand radiating, as that whereby one is heated, consumed, scorched,\nwhereby that which has been eaten, drunk, chewed, or tasted, is\nfully digested; or whatever other dependent properties, which on one\'s\nown person and body are heating and radiating this is called one\'s own\nheating element. Now, whether it be one\'s own heating element, or\nwhether it be the external heating element, they are both only the\nheating element.\n  And one should understand, according to reality, and true wisdom:\n"This does not belong to me; this am I not; this is not my E

In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = text.replace("\n", " ") #remove newlines

    # remove roman numerals / standalone numbers / weird chars
    text = re.sub(r'\b[ivxlcdm]+\b', ' ', text)  # roman numerals
    text = re.sub(r'[^a-z\s]', ' ', text)        # keep only letters + spaces

    # collapse multiple spaces into one
    text = re.sub(r'\s+', ' ', text).strip()

    text = re.sub(r"\b\d+\b", " ", text) #remove all the digits
    text = re.sub(r"[^a-z\s]", " ", text) #keep only letters & spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
start_index = corpus.find('INTRODUCTION')
corpus = corpus.replace("\n", " ")
corpus = corpus[start_index : ]

In [ ]:
# tokenize into sentences
sentences = sent_tokenize(corpus)

# keep only sentences where all words are lowercase
lowercase_sentences = []

for s in sentences:
  words = re.findall(r"\b\w+\b", s) #get words
  if all(w.islower() for w in words):
    lowercase_sentences.append(s)

In [ ]:
lowercase_sentences

['cries the priest.',
 'says the learned doctor.',
 'yet it made them faulter and hesitate more in maintaining their principles, and was even apt to produce, in certain dispositions of mind, some practices and opinions, which had the appearance of determined infidelity.',
 '190b.',
 '), maintain that there is no boy or old woman so ridiculous as to believe the poets in their accounts of a future state.',
 'there is no longer a remedy.',
 'or essential feature.',
 '; and those who were to follow the profession of hieroglyphical painting were taught the application of the characters appropriated to each of these branches.',
 'changes of the seasons with those of the place of the sun at his rising and setting.',
 'he inquired of a young peasant who was unacquainted with his person.',
 'rejoined the prince.',
 'for the foreign ambassadors; and a spacious saloon, with apartments: opening into it, for men of science and poets, who pursued their studies in this retreat, or met together to hol

In [ ]:
import re

def clean_sentences(sent_list, min_words=2):
    cleaned = []
    for s in sent_list:

        # remove punctuation
        s = re.sub(r"[^\w\s]", "", s)


        # remove leading/trailing quotes and dots
        s = s.strip('"').strip("'").strip()
        s = re.sub(r"\.\.\.", "", s)   # remove "..."

        # remove numbers at start of words
        s = re.sub(r"\b\d+", "", s)


        # replace multiple types of quotes inside text
        s = s.replace('“', '"').replace('”', '"').replace('‘', "'").replace('’', "'")



        # ensure sentence ends with . ? or !
        if not re.search(r"[.!?]$", s):
            s += "."

        # wrap sentence in quotes
        s = f'"{s}"'


        # skip too-short fragments
        if len(s.split()) >= min_words:
            cleaned.append(s)

    return cleaned



cleaned_sentences = clean_sentences(lowercase_sentences)

for s in cleaned_sentences:
  print(s)


"cries the priest."
"says the learned doctor."
"yet it made them faulter and hesitate more in maintaining their principles and was even apt to produce in certain dispositions of mind some practices and opinions which had the appearance of determined infidelity."
"maintain that there is no boy or old woman so ridiculous as to believe the poets in their accounts of a future state."
"there is no longer a remedy."
"or essential feature."
"and those who were to follow the profession of hieroglyphical painting were taught the application of the characters appropriated to each of these branches."
"changes of the seasons with those of the place of the sun at his rising and setting."
"he inquired of a young peasant who was unacquainted with his person."
"rejoined the prince."
"for the foreign ambassadors and a spacious saloon with apartments opening into it for men of science and poets who pursued their studies in this retreat or met together to hold converse under its marble porticos."
"from t

In [ ]:
corpus = cleaned_sentences

In [ ]:
# next, joining them into a single text string (needed for tokenization)
corpus = " ".join([s.strip('"') for s in cleaned_sentences])

In [ ]:
# converting the corpus into a sequence of tokens
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
total_words = len(tokenizer.word_index) + 1 #because keras indexing starts from 1

print(f"Total Unique words : {total_words}")

Total Unique words : 6265


corpus is assumed to be a string of text.



.split() split it into a list of words(corpus_words)



example -> "I am Taru" -> ["I", "am", "Taru"]



input_sequences = [] just initializes an empty list to store sequences



the loop goes from i=1 to the last index of corpus_words.

corpus_words[:i+1] creates a sublist from the start to the i-th word.

these sublists are called n-gram sequences because each one contains a growing number of words from the start.



example of input_sequence is
[
  ['I', 'am'],
  ['I', 'am', 'an'],
  ['I', 'am', 'an', 'engineering'],
  ['I', 'am', 'an', 'engineering', 'student],
]

In [ ]:
# find max sequence length
max_sequence_length = 20

In [ ]:
input_sequences = []
corpus_words = corpus.split()

# create sequences like [w1, w2, w3] -> w4

for i in range(1, len(corpus_words)):
  start_idx = max(0, i - max_sequence_length)
  n_gram_sequence = corpus_words[start_idx : i+1]
  if(len(n_gram_sequence) < 2):
    continue
  input_sequences.append(n_gram_sequence)

In [ ]:
input_sequences[:3]

[['cries', 'the'],
 ['cries', 'the', 'priest.'],
 ['cries', 'the', 'priest.', 'says']]

In [ ]:
input_sequences_int = []

for seq in input_sequences:
  token_seq = [tokenizer.word_index[word] for word in seq if word in tokenizer.word_index]
  if(len(token_seq) < 2):
    continue
  input_sequences_int.append(token_seq)

pad sequences

In [ ]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np


x = []
y = []


for seq in input_sequences_int:
    if len(seq) < 2:
      continue

    x.append(seq[:-1])  # all except last word
    y.append(seq[-1])   # last word is label

# pad X once
x = pad_sequences(x, maxlen=max_sequence_length, padding='pre')
y = np.array(y)

In [ ]:
x[:1]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0, 1292]],
      dtype=int32)

split predictors and label

predict the last word of each sequence from the previous words

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Input


model = Sequential()
# input_dim = total_words : vocabulary size(number of unique words)

# output_dim = how many dimensions you want to represent each word
# represent each word as a vector with 100 coordinates

# input_length = x.shape[1], how long each input sequence is(after padding)


model.add(Embedding(input_dim=total_words, output_dim=50, input_length=x.shape[1]))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1758489411.575980      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758489411.576749      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Force model to build with expected input shape
model.build(input_shape=(None, x.shape[1]))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 50)         │       313,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 20, 256)        │       314,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6265)           │     1,610,105 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,763,035 (10.54 MB)

 Trainable params: 2,763,035 (10.54 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(x, y, epochs=50, batch_size=16)

Epoch 1/50


I0000 00:00:1758489417.562414      85 cuda_dnn.cc:529] Loaded cuDNN version 90300


3620/3620 ━━━━━━━━━━━━━━━━━━━━ 32s 7ms/step - accuracy: 0.0609 - loss: 6.5387
Epoch 2/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - accuracy: 0.0858 - loss: 6.0498
Epoch 3/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.0999 - loss: 5.8843
Epoch 4/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.1118 - loss: 5.6534
Epoch 5/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.1265 - loss: 5.4279
Epoch 6/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.1404 - loss: 5.1135
Epoch 7/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.1558 - loss: 4.8041
Epoch 8/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - accuracy: 0.1879 - loss: 4.4291
Epoch 9/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - accuracy: 0.2173 - loss: 4.1014
Epoch 10/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.2601 - loss: 3.7348
Epoch 11/50
3620/3620 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.3045 - loss: 3.4000
Epoch 12/50
3620/3620 ━━━━━━━━

In [ ]:
def predict_next_word(model, tokenizer, text_seq, max_seq_len, temperature=1.0, top_k=5):
    token_list = tokenizer.texts_to_sequences([text_seq])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')

    predicted = model.predict(token_list, verbose=0)[0]

    # apply temperature
    preds = np.log(predicted + 1e-10) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # pick from top-k words
    top_indices = preds.argsort()[-top_k:][::-1]
    top_probs = preds[top_indices]
    top_probs /= top_probs.sum()
    predicted_word_index = np.random.choice(top_indices, p=top_probs)

    return tokenizer.index_word.get(predicted_word_index, "")


In [ ]:
seed_text = "i am just an innocent girl"

for i in range(1, 50):
    next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_length)
    print(f"{seed_text} {next_word}")
    seed_text = seed_text + " " +  next_word


i am just an innocent girl is
i am just an innocent girl is the
i am just an innocent girl is the let
i am just an innocent girl is the let him
i am just an innocent girl is the let him let
i am just an innocent girl is the let him let him
i am just an innocent girl is the let him let him sing
i am just an innocent girl is the let him let him sing with
i am just an innocent girl is the let him let him sing with though
i am just an innocent girl is the let him let him sing with though a
i am just an innocent girl is the let him let him sing with though a will
i am just an innocent girl is the let him let him sing with though a will not
i am just an innocent girl is the let him let him sing with though a will not you
i am just an innocent girl is the let him let him sing with though a will not you will
i am just an innocent girl is the let him let him sing with though a will not you will be
i am just an innocent girl is the let him let him sing with though a will not you will be no
i am 